# Import Libraries

In [ ]:
from __future__ import absolute_import, division, print_function
import os
import base64
import matplotlib
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
import pickle
import pandas_gbq
import random
from tqdm.notebook import tnrange
from ipywidgets import IntProgress
from sklearn.preprocessing import MinMaxScaler
import joblib

# Print current working directory
print("Current working dir : %s" % os.getcwd())

# Importing Data

In [ ]:
base_filename = 'ICU_SEPSIS3'
query = """SELECT * FROM `vibrant-shell-313523.MIMIC_IV_v1.ICU_SEPSIS3_MOD`"""
hours_prev = 24
regen = False

In [ ]:
################################### ICU Reduced Filtered dataset #######################################################
if not os.path.isfile(os.path.join(os.getcwd(),(base_filename +'_ALL.pickle'))) or regen:
    df = pandas_gbq.read_gbq(query, project_id="vibrant-shell-313523", use_bqstorage_api=True, progress_bar_type='tqdm')
    
    with open(os.path.join(os.getcwd(),(base_filename +'_ALL.pickle')), 'wb') as f:
        pickle.dump(df, f)
else:
    with open(os.path.join(os.getcwd(),(base_filename +'_ALL.pickle')), 'rb') as f:
        df = pickle.load(f)
print(f'Dataset loaded.')

# Pre-Processing Data

In [ ]:
# Remove columns with 100% missing values
df = df[df.columns[df.isnull().mean() != 1.0]]

In [ ]:
############### Sample of dataset ####################
df.sort_values(by=['stay_id','hr'], axis=0, ascending=True, inplace=True, ignore_index=True)
df.tail()

In [ ]:
df_stay = df.stay_id.unique()
dictOfIDs = { i : np.where(df_stay == i)[0][0] for i in df_stay}

# Remap the values of the dataframe
df["PatientID"] = df["stay_id"].map(dictOfIDs)

###### Sample of dataset #######
display(df.head())

In [ ]:
df.isna().sum().sum()

## Shifting labels 6 or 12 hours ahead

In [ ]:
if hours_prev != 0:
    label_shifted = pd.DataFrame(df.groupby('PatientID')['SepsisLabel'].shift(-hours_prev).ffill().astype(int), columns=['SepsisLabel'])
    df_dropped_labels = df.drop(columns=['SepsisLabel'])
    df = pd.concat([df_dropped_labels,label_shifted],axis=1)

## Forward Filling Weight, Age, Ethnicity and Gender (Administrative Variables)

In [ ]:
def group_column_forward_fill(df,columns):
    df_seg = df[columns].copy(deep=True)
    columns.remove('PatientID')
    columns.remove('hr')
    df_dropped = df.drop(columns=columns)
    df_ff = df_seg.groupby('stay_id').ffill()
    df_ff_merged = pd.merge(df_dropped,df_ff, how='left', left_on=['PatientID','hr'], right_on = ['PatientID','hr'])
    return df_ff_merged.reset_index(drop=True)

In [ ]:
admin_feats = list(['stay_id','weight','admission_age','gender','PatientID','hr'])
df_f = group_column_forward_fill(df, admin_feats)

In [ ]:
df_f.isna().sum().sum()

# Encoding Categorical Data

In [ ]:
# Categorical Data
object_list = []
for column in df_f.columns:
    if df_f[column].dtype == object or df_f[column].dtype == bool:
        # print(f"{column} with unique type {df[column].unique()}")
        object_list.append(column)
df_cat = df_f[object_list + ['PatientID','hr']]
df_cat.head()

In [ ]:
# Encoding the variable
df_cat_enc = pd.get_dummies(df_cat, columns=object_list, dummy_na=True)

In [ ]:
df_cat_enc.head()

# Extracting only values in clinical ranges

In [ ]:
# Numerical Data, Excludind Label and Patient ID
df_num = df_f[df_f.columns.difference(object_list)].copy(deep=True)
df_num.head()

In [ ]:
range_df = pd.read_excel(os.path.join(os.getcwd(),'clinical_ranges.xlsx')).set_index('Column Name')
display(range_df)

In [ ]:
def remove_outliers(df, range_df,percent_change=0.75):
    df_new = df.copy(deep=True)
    for column in df[df.columns.difference(['SepsisLabel', 'PatientID',  'hr', 'stay_id','SOFA_24h','los_hospital'])].columns:
        # Calculate true limits
        try:
            max = pd.to_numeric(range_df.loc[column,'Maximum'].max())
            min = pd.to_numeric(range_df.loc[column,'Minimum'].min())
        except:
            max = pd.to_numeric(range_df.loc[column,'Maximum'])
            min = pd.to_numeric(range_df.loc[column,'Minimum'])
        deviation = (max-min)*percent_change
        if min == 0: 
            low_limit = 0
        else:
            low_limit = min - deviation
        high_limit = max + deviation
        # Apply filter with respect to IQR, including optional whiskers
        filter = (df[column] >= low_limit) & (df[column] <= high_limit)
        percent_retained = (df[column].between(low_limit,high_limit).sum()/df[column].count())*100
        print('Keep',percent_retained,'%\ of',column)
        df_new[column] = df[column].loc[filter]
    return df_new

In [ ]:
df_num_mod = remove_outliers(df_num, range_df, percent_change=0.90)

In [ ]:
df_num_mod.isna().sum().sum()

In [ ]:
df_num_mod.isna().sum().sum() - df_num.isna().sum().sum()

# Scaling numerical values

In [ ]:
######################## Data Normalization #########################################
scaler = MinMaxScaler((0,1))

# Copy dataframe to just substitute later
df_num_scaled = df_num_mod.copy(deep=True)

# Extract only the columns to be scaled  
df_num_s = df_num_mod[df_num_mod.columns.difference(['SepsisLabel', 'PatientID',  'hr', 'stay_id','SOFA_24h'])]

# Scale the data
df_num_scaled[df_num_mod.columns.difference(['SepsisLabel', 'PatientID',  'hr', 'stay_id','SOFA_24h'])] = scaler.fit_transform(df_num_s)

# Save scaler values to import after testing
if hours_prev != 0:
    scaler_filename = "scaler_onset_pre_" + str(hours_prev) + "hr.pickle"
else:
    scaler_filename = "scaler_onset.pickle"

joblib.dump(scaler, scaler_filename)

Feat_Min_Max = pd.DataFrame({'Feature':df_num_s.columns, 'Min':scaler.data_min_, 'Max':scaler.data_max_})
Feat_Min_Max.to_csv(os.path.join(os.getcwd(),'Feature_Min_Max.csv'))

# Display Min and Max
display(Feat_Min_Max)

# Join and save Encoded, Scaled Dataset

In [ ]:
df_enc_scaled = pd.merge(df_num_scaled, df_cat_enc, how='left', left_on=['PatientID','hr'], right_on = ['PatientID','hr'])
df_enc_scaled.head()

In [ ]:
if hours_prev !=0:
    ################################### ICU Onset Pre-Processed Dataset #######################################################
    with open(os.path.join(os.getcwd(),(base_filename+'_ENC_SCALED_PRE_'+ str(hours_prev) +'HR.pickle')), 'wb') as f:
        pickle.dump(df_enc_scaled, f)
else:
    ################################### ICU Onset Pre-Processed Dataset #######################################################
    with open(os.path.join(os.getcwd(),(base_filename +'_ENC_SCALED.pickle')), 'wb') as f:
        pickle.dump(df_enc_scaled, f)

print(f'Dataset Saved with labels ',hours_prev,' hours pervious to diagnosis')

# Step 2: Imputation Strategy

## Interpolating values per patient

In [ ]:
df_num_scaled_inter = df_num_scaled.groupby('PatientID').apply(lambda group: group.interpolate(limit_area='inside')).reset_index(drop=True)
df_num_scaled_inter.head()

## Join and Save Encoded, Scaled and Interpolated Dataset

In [ ]:
df_enc_scaled_inter = pd.merge(df_num_scaled_inter, df_cat_enc, how='left', left_on=['PatientID','hr'], right_on = ['PatientID','hr'])
df_enc_scaled_inter.head()

In [ ]:
if hours_prev !=0:
    ################################### ICU Onset Pre-Processed Dataset #######################################################
    with open(os.path.join(os.getcwd(),(base_filename+'_ENC_SCALED_INTER_'+ str(hours_prev) +'HR.pickle')), 'wb') as f:
        pickle.dump(df_num_scaled_inter, f)
else:
    ################################### ICU Onset Pre-Processed Dataset #######################################################
    with open(os.path.join(os.getcwd(),(base_filename +'_ENC_SCALED_INTER.pickle')), 'wb') as f:
        pickle.dump(df_num_scaled_inter, f)

print(f'Dataset Saved with labels ',hours_prev,' hours pervious to diagnosis')

# Step 3: Imputation Strategy

## Stratification by Admission SOFA

In [ ]:
df_grouped = df_enc_scaled_inter.groupby(['hr'])

In [ ]:
init_SOFA = df_grouped.get_group(1)[['PatientID','SOFA_24h']].reset_index(drop=True)

In [ ]:
init_SOFA_group = init_SOFA.groupby('SOFA_24h')

## Fill Forwards/median imputation

In [ ]:
df_list = []
df_pat = pd.DataFrame(columns=df_enc_scaled_inter.columns)
for key,item in init_SOFA_group:
  patients_SOFA = init_SOFA_group.get_group(key)['PatientID'].values
  patients_SOFA_data = df_enc_scaled_inter.loc[df_enc_scaled_inter['PatientID'].isin(patients_SOFA)]
  print('Missing before imputation:',patients_SOFA_data.isna().sum().sum())
  
  cols = []
  for col in patients_SOFA_data:
    if patients_SOFA_data[col].std() > 0.1 and patients_SOFA_data[col].std() <= 1:
      cols.append(col)

  patients_SOFA_ffill = patients_SOFA_data.groupby('PatientID').ffill()[cols]
  patients_SOFA_median = patients_SOFA_data[patients_SOFA_data.columns.difference(cols)].fillna(patients_SOFA_data[patients_SOFA_data.columns.difference(cols)].median())#.reset_index(drop=True)
  patients_SOFA_data_filled = pd.concat([patients_SOFA_ffill,patients_SOFA_median],axis=1)

  print('Missing after imputation:',patients_SOFA_data_filled.isna().sum().sum())
  df_list.append(patients_SOFA_data_filled)

full_df = pd.concat(df_list).reset_index(drop=True)

# Save Encoded, Scaled, Interpolated and Foward filled Dataset

In [ ]:
full_df.head()

In [ ]:
if hours_prev !=0:
    ################################### ICU Onset Pre-Processed Dataset #######################################################
    with open(os.path.join(os.getcwd(),(base_filename+'_ENC_SCALED_INTER_FFILLorMEDIAN_'+ str(hours_prev) +'HR.pickle')), 'wb') as f:
        pickle.dump(full_df, f)
else:
    ################################### ICU Onset Pre-Processed Dataset #######################################################
    with open(os.path.join(os.getcwd(),(base_filename +'_ENC_SCALED_INTER_FFILLorMEDIAN.pickle')), 'wb') as f:
        pickle.dump(full_df, f)

print(f'Dataset Saved with labels ',hours_prev,' hours pervious to diagnosis')